# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
#https://iexcloud.io/docs/api/

import pandas as pd
import numpy as np
import matplotlib as plt
import xlsxwriter  
import requests
import math #The Python math module

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from Api_token import IEX_CLOUD_API_TOKEN


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol= "AAPL"
Api_url= f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}" # sandbox url! with quote endpoint( this is art for iexcloud )
data = requests.get(Api_url).json()






## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
market_cap = data["marketCap"]
price = data["latestPrice"]
print(market_cap,price)

2281123156305 138.33


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_column= ['Ticker','Price','Market Cap','Shares to Buy']
final_data = pd.DataFrame(columns= my_column)
final_data

,Ticker,Price,Market Cap,Shares to Buy


In [7]:
final_data.append(pd.Series([symbol,price,market_cap,'N/A'],
            index = my_column), ignore_index = True)


,Ticker,Price,Market Cap,Shares to Buy
0,AAPL,138.33,2281123156305,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_data= pd.DataFrame(columns= my_column)

for i in stocks["Ticker"][:5]:

    Api_url= f"https://sandbox.iexapis.com/stable/stock/{i}/quote/?token={IEX_CLOUD_API_TOKEN}" # sandbox url! with quote endpoint( this is art for iexcloud )
    data = requests.get(Api_url).json()
    final_data= final_data.append(pd.Series(
        [i,data["latestPrice"],data["marketCap"],"N/A"], index= my_column),
        ignore_index= True)
   

    
 


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst.""" 
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        

In [10]:
symbol_groups= list(chunks(stocks["Ticker"],100))# 
symbol_string=[]

for i in range(len(symbol_groups)):
    symbol_string.append(",".join(symbol_groups[i]))

final_data= pd.DataFrame(columns=my_column)

for i in symbol_string:
    Batch_Api_url= f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={i}&token={IEX_CLOUD_API_TOKEN}" # sandbox url! with quote endpoint( this is art for iexcloud )
    
    data = requests.get(Batch_Api_url).json()
   
    for a in i.split(","):
      final_data= final_data.append( pd.Series([a, 
                                                   data[a]['quote']['latestPrice'], 
                                                   data[a]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_column), 
                                        ignore_index = True)
final_data

    
        
    
    



,Ticker,Price,Market Cap,Shares to Buy
0,A,123.67,38234330566,N/A
1,AAL,17.62,10415940840,N/A
2,AAP,152.56,10285337603,N/A
3,AAPL,134.17,2326061968063,N/A
4,ABBV,105.18,184877052033,N/A
...,...,...,...,...
500,YUM,102.09,31127026397,N/A
501,ZBH,153.83,32184608155,N/A
502,ZBRA,405.58,20815645807,N/A
503,ZION,45.59,7391680081,N/A


In [11]:
for i in symbol_string:
    

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [11]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:

    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100000


In [13]:
position_size = float(portfolio_size) / len(final_data.index)
print(position_size)
for i in range(0, len(final_data['Ticker'])-1):
  final_data.loc[i,'Shares to Buy']= math.floor(position_size/final_data["Price"][i])

final_data



198.01980198019803


,Ticker,Price,Market Cap,Shares to Buy
0,A,119.90,37049955798,1
1,AAL,16.92,10497172939,11
2,AAP,150.90,10075750194,1
3,AAPL,148.79,2491520047034,1
4,ABBV,102.83,190488005414,1
...,...,...,...,...
500,YUM,104.47,31057025232,1
501,ZBH,154.90,33198622700,1
502,ZBRA,402.03,21734077730,0
503,ZION,45.73,7503785291,4


In [21]:
for i in range(0, len(final_data['Ticker'])-1):
  print(final_data.loc[i,'Shares to Buy'])


N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A


In [22]:
for i in range(0, len(final_data['Ticker'])-1):
  print(final_data['Shares to Buy'])

0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, 

Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N

Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N/A
504    N/A
Name: Shares to Buy, Length: 505, dtype: object
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
500    N/A
501    N/A
502    N/A
503    N

In [15]:
for i in range(0, len(final_data['Ticker'])-1):
  final_data.loc[i,'Shares to Buy']= math.floor(position_size/final_data.loc[i,["Price"]])
# u dont need to call the api to acces data again, it is already in ur systems data frame! thus just call the right columns ext.

final_data

,Ticker,Price,Market Cap,Shares to Buy
0,A,119.90,37049955798,1
1,AAL,16.92,10497172939,11
2,AAP,150.90,10075750194,1
3,AAPL,148.79,2491520047034,1
4,ABBV,102.83,190488005414,1
...,...,...,...,...
500,YUM,104.47,31057025232,1
501,ZBH,154.90,33198622700,1
502,ZBRA,402.03,21734077730,0
503,ZION,45.73,7503785291,4


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [87]:
writer = pd.ExcelWriter("Recommended_Trades_To_Execute.xlsx",engine= "xlsxwriter")
final_data.to_excel(writer,"Recommended Trades To Execute",index= False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [88]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [89]:
# writer.sheets["Recommended Trades To Execute"].set_column('A:A',18,string_format)
# writer.sheets["Recommended Trades To Execute"].set_column('B:B',18,string_format)
# writer.sheets["Recommended Trades To Execute"].set_column('C:C',18,string_format)
# writer.sheets["Recommended Trades To Execute"].set_column('D:D',18,string_format)
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# and fior the HEADERS!!!
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [90]:
column_format={
    "A":["Ticker",string_format],
    "B":["Price",dollar_format],
    "C":["Market Cap",dollar_format],
    "D":["Shares to Buy", integer_format]
}

for key,value in column_format.items():
    writer.sheets["Recommended Trades To Execute"].set_column(f'{key}:{key}',18,value[1])
    writer.sheets['Recommended Trades To Execute'].write(f'{key}1', value[0], string_format) # cuz they are all strings!!

## Saving Our Excel Output

Saving our Excel file is very easy:

In [91]:
writer.save()